<a href="https://colab.research.google.com/github/yundongny1/personal-project/blob/main/final_model_ctr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Spark

In [ ]:
#@title  <-- Run Me: (Hidden) Installing Spark
%%bash 

## Setup Spark on Colab
pip install -q pyspark
apt-get -qq install -y openjdk-8-jdk-headless

## Setup port-forwarding

# Download ngrok
wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# Unload ngrok
unzip -q ngrok-stable-linux-amd64.zip

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting up ope

In [ ]:
#@title  <-- Run Me: (Hidden) Environment Variable Setup
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pyspark 
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd    
import numpy as np


# Set configuration scope to be local and use port 4050
config_scope = SparkConf().set("spark.ui.port", "4050")

# Create the connection to a Spark cluster
sc = pyspark.SparkContext(conf = config_scope)

# Create a session to programmatically build Spark RDD, DataFrame and DataSet
spark = SparkSession.builder.getOrCreate()

## Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read in data
TRAIN_DIR = "drive/MyDrive/STAT480-Group Project/dataset/track2"

In [ ]:
training = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/training.txt')
training = training.selectExpr('_c0 as Click', '_c1 as Impression', '_c2 as AdURL', '_c3 as AdId', '_c4 as AdvId', 
                    '_c5 as Depth', '_c6 as Pos', '_c7 as QId', '_c8 as KeyId', '_c9 as TitleId', 
                    '_c10 as DescId', '_c11 as UId')
training.show(10)

+-----+----------+--------------------+--------+-----+-----+---+--------+------+-------+-------+------+
|Click|Impression|               AdURL|    AdId|AdvId|Depth|Pos|     QId| KeyId|TitleId| DescId|   UId|
+-----+----------+--------------------+--------+-----+-----+---+--------+------+-------+-------+------+
|    0|         1| 4298118681424644510| 7686695|  385|    3|  3|    1601|  5521|   7709|    576|490234|
|    0|         1| 4860571499428580850|21560664|37484|    2|  2| 2255103|   317|  48989|  44771|490234|
|    0|         1| 9704320783495875564|21748480|36759|    3|  3| 4532751| 60721| 685038|  29681|490234|
|    0|         1|13677630321509009335| 3517124|23778|    3|  1|    1601|  2155|   1207|   1422|490234|
|    0|         1| 3284760244799604489|20758093|34535|    1|  1| 4532751| 77819| 266618| 222223|490234|
|    0|         1|10196385171799537224|21375650|36832|    2|  1| 4688625|202465| 457316| 429545|490234|
|    0|         1| 4203081172173603803| 4427028|28647|    3|  1|

In [ ]:
purchasekeywordid = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/purchasedkeywordid_tokensid.txt')
purchasekeywordid = purchasekeywordid.selectExpr('_c0 as KeyId', '_c1 as PurchaseKeyword')
purchasekeywordid.show(10)

+-----+---------------+
|KeyId|PurchaseKeyword|
+-----+---------------+
|    0|          12731|
|    1|           1545|
|    2|            477|
|    3|     1545|75|31|
|    4|            279|
|    5|           2684|
|    6|             27|
|    7|     4189|75|31|
|    8|        110|298|
|    9|             30|
+-----+---------------+
only showing top 10 rows



In [ ]:
queryid = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/queryid_tokensid.txt')
queryid = queryid.selectExpr('_c0 as QId', '_c1 as Query')
queryid.show(10)

+---+--------------+
|QId|         Query|
+---+--------------+
|  0|         12731|
|  1|    1545|75|31|
|  2|           383|
|  3|      518|1996|
|  4|    4189|75|31|
|  5|          1545|
|  6|           279|
|  7|          2684|
|  8|           143|
|  9|4189|75|31|290|
+---+--------------+
only showing top 10 rows



In [ ]:
descriptionid = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/descriptionid_tokensid.txt')
descriptionid = descriptionid.selectExpr('_c0 as DescId', '_c1 as Description')
descriptionid.show(10)

+------+--------------------+
|DescId|         Description|
+------+--------------------+
|     0|1545|31|40|615|1|...|
|     1|172|46|467|170|56...|
|     2|2672|6|1159|10966...|
|     3|13280|35|1299|26|...|
|     4|13327|99|128|494|...|
|     5|12731|390|1354|1|...|
|     6|1666|1|277|198|12...|
|     7|69893|1894|2|1188...|
|     8|2389|6|15|110|562...|
|     9|13280|35|1299|26|...|
+------+--------------------+
only showing top 10 rows



In [ ]:
userid = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/userid_profile.txt')
userid = userid.selectExpr('_c0 as UId', '_c1 as Gender', '_c2 as Age')
userid.show(10)

+---+------+---+
|UId|Gender|Age|
+---+------+---+
|  1|     1|  5|
|  2|     2|  3|
|  3|     1|  5|
|  4|     1|  3|
|  5|     2|  1|
|  6|     2|  3|
|  7|     2|  3|
|  8|     2|  4|
|  9|     1|  3|
| 10|     2|  5|
+---+------+---+
only showing top 10 rows



In [ ]:
titleid = spark.read.option("header","false").option("delimiter","\t").csv(TRAIN_DIR+'/titleid_tokensid.txt')
titleid = titleid.selectExpr('_c0 as TitleId', '_c1 as Titile')

In [ ]:
titleid.show(10)

+-------+--------------------+
|TitleId|              Titile|
+-------+--------------------+
|      0|615|1545|75|31|1|...|
|      1|466|582|685|1|42|...|
|      2|12731|190|513|127...|
|      3|2371|3970|1|2805|...|
|      4|165|134|460|2887|...|
|      5|2389|6|0|203|616|...|
|      6|449|144880|8044|3...|
|      7| 51|0|12731|8|183|94|
|      8|35|2233|1024|455|...|
|      9|685|282|310|1|138...|
+-------+--------------------+
only showing top 10 rows



## Data pre-processing

### Gender and Age

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder

In [ ]:
# label encode
Gender_indexer = StringIndexer(inputCol='Gender', outputCol='Gender_num').fit(userid)
Age_indexer = StringIndexer(inputCol='Age', outputCol='Age_num').fit(userid)


userid = Gender_indexer.transform(userid)
userid = Age_indexer.transform(userid)

In [ ]:
# one-hot encoder
Gender_onehoter = OneHotEncoder(inputCol='Gender_num', outputCol='Gender_vector')
Age_onehoter = OneHotEncoder(inputCol='Age_num', outputCol='Age_vector')


ohe1 = Gender_onehoter.fit(userid)
ohe2 = Age_onehoter.fit(userid)


userid = ohe1.transform(userid)
userid = ohe2.transform(userid)

In [ ]:
userid.show(10)

+---+------+---+----------+-------+-------------+-------------+
|UId|Gender|Age|Gender_num|Age_num|Gender_vector|   Age_vector|
+---+------+---+----------+-------+-------------+-------------+
|  1|     1|  5|       0.0|    3.0|(2,[0],[1.0])|(5,[3],[1.0])|
|  2|     2|  3|       1.0|    0.0|(2,[1],[1.0])|(5,[0],[1.0])|
|  3|     1|  5|       0.0|    3.0|(2,[0],[1.0])|(5,[3],[1.0])|
|  4|     1|  3|       0.0|    0.0|(2,[0],[1.0])|(5,[0],[1.0])|
|  5|     2|  1|       1.0|    4.0|(2,[1],[1.0])|(5,[4],[1.0])|
|  6|     2|  3|       1.0|    0.0|(2,[1],[1.0])|(5,[0],[1.0])|
|  7|     2|  3|       1.0|    0.0|(2,[1],[1.0])|(5,[0],[1.0])|
|  8|     2|  4|       1.0|    1.0|(2,[1],[1.0])|(5,[1],[1.0])|
|  9|     1|  3|       0.0|    0.0|(2,[0],[1.0])|(5,[0],[1.0])|
| 10|     2|  5|       1.0|    3.0|(2,[1],[1.0])|(5,[3],[1.0])|
+---+------+---+----------+-------+-------------+-------------+
only showing top 10 rows



### Average Click Through Rate

In [ ]:
# average click through rate
temp_df1 = training.groupBy("AdvId").agg((avg("Click")/count("Click")).alias("AvgClick_Advertiser"))
temp_df1.show(10)

+-----+--------------------+
|AdvId| AvgClick_Advertiser|
+-----+--------------------+
| 6240|2.655942187761288...|
|16250|2.414313518808267E-6|
|16576|5.577307635578552E-6|
|18509|3.446611080489074E-5|
|15574| 5.67589982843738E-6|
| 3959|3.766468677563665...|
|35640|4.284214261725126E-6|
|36067|2.438973812874274...|
| 1512|7.907478035101003E-7|
| 9993|1.491296890604827...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
temp_df2 = training.groupBy("AdID").agg((avg("Click")/count("Click")).alias("AvgClick_Ad"))
temp_df2.show(10)

+--------+--------------------+
|    AdID|         AvgClick_Ad|
+--------+--------------------+
| 3373959|6.114538064567283E-5|
| 4331513|4.759071980963712...|
|22158484|7.941253056966983E-8|
|21405511|3.327416385954420...|
|21161676|3.228305785123967E-5|
|20563853|5.245001088692118E-6|
|21697120|1.143118427069044...|
|22134864|                 0.0|
|20935476|1.502313562886845...|
|20665592|3.218520887928685...|
+--------+--------------------+
only showing top 10 rows



In [ ]:
temp_df3 = training.groupBy("QId").agg((avg("Click")/count("Click")).alias("AvgClick_Query"))
temp_df3.show(10)

+--------+--------------------+
|     QId|      AvgClick_Query|
+--------+--------------------+
|    7252|3.282448093554147E-5|
|19195250|                 0.0|
|  251284| 5.94883997620464E-4|
|  497229|                 0.0|
|   11563|2.409557267947587E-5|
| 1175624|                 0.0|
|    2162|4.856653814673367...|
|   55089|2.602617489589530...|
|24924868|                 0.0|
|17640726|                 0.0|
+--------+--------------------+
only showing top 10 rows



In [ ]:
temp_df4 = training.groupBy("KeyId").agg((avg("Click")/count("Click")).alias("AvgClick_Key"))
temp_df4.show(10)

+------+--------------------+
| KeyId|        AvgClick_Key|
+------+--------------------+
| 30002|3.459000752332664E-5|
|  1512|4.175182556630501E-6|
| 23318|2.029056083110137E-5|
|  3210|1.335024194128823...|
|   296|  9.8701362446419E-7|
|  2294|5.153212629260496...|
|  2069|2.549041550331591...|
|671956|                 0.0|
| 17401|1.170516859882673...|
| 39645|1.334659904389818E-4|
+------+--------------------+
only showing top 10 rows



In [ ]:
temp_df5 = training.groupBy("TitleId").agg((avg("Click")/count("Click")).alias("AvgClick_Title"))
temp_df5.show(10)

+-------+--------------------+
|TitleId|      AvgClick_Title|
+-------+--------------------+
|    829|3.876514672414457E-6|
| 451192|0.003567181926278...|
|   4937|5.283697703680402E-5|
| 231563|                 0.0|
| 147414|4.058112166220274E-4|
|   2069|7.873862574234313E-6|
|  16576|1.555263166080332...|
| 271864|3.844675124951941...|
|    675|9.207285107188607E-6|
|  19095|4.535402943064201E-5|
+-------+--------------------+
only showing top 10 rows



In [ ]:
temp_df6 = training.groupBy("DescId").agg((avg("Click")/count("Click")).alias("AvgClick_Desciption"))
temp_df6.show(10)

+------+--------------------+
|DescId| AvgClick_Desciption|
+------+--------------------+
|973354|                 0.0|
|  3414|1.129513499508115...|
|216438|0.001423994304022...|
|232050|                 0.0|
| 27108|5.519628904309505E-5|
|   829|2.645304194908614E-6|
|256062|                 0.0|
|   691| 1.12654217723878E-6|
|   467|2.656038998339539E-6|
| 58185|2.085027418110548E-5|
+------+--------------------+
only showing top 10 rows



### Join tables together

In [ ]:
train_df = training.join(userid, on="UId", how="inner")
train_df = train_df.join(temp_df1, on="AdvId", how="inner")
train_df = train_df.join(temp_df2, on="AdId", how="inner")
train_df = train_df.join(temp_df3, on="QId", how="inner")
train_df = train_df.join(temp_df4, on="KeyId", how="inner")
train_df = train_df.join(temp_df5, on="TitleId", how="inner")
train_df = train_df.join(temp_df6, on="DescId", how="inner")
train_df.show(10)

+-------+-------+-----+--------+--------+-----+--------+-----+----------+--------------------+-----+---+------+---+----------+-------+-------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
| DescId|TitleId|KeyId|     QId|    AdId|AdvId|     UId|Click|Impression|               AdURL|Depth|Pos|Gender|Age|Gender_num|Age_num|Gender_vector|   Age_vector| AvgClick_Advertiser|         AvgClick_Ad|AvgClick_Query|        AvgClick_Key|      AvgClick_Title| AvgClick_Desciption|
+-------+-------+-----+--------+--------+-----+--------+-----+----------+--------------------+-----+---+------+---+----------+-------+-------------+-------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|1000096|1094377| 8989| 2181703| 4838685|19509|    4818|    0|         1| 3902477925057104899|    2|  2|     1|  3|       0.0|    0.0|(2,[0],[1.0])|(5,

## Logistic Regression Model

In [ ]:
from pyspark.ml.feature import VectorAssembler

train_assembler = VectorAssembler(inputCols=['Impression', 'Depth', 'Pos', 
                        'Gender_vector', 'Age_vector',
                        'AvgImp_Advertiser', 'AvgClick_Ad',
                        'AvgClick_Query', 'AvgClick_Key',
                        'AvgClick_Title', 'AvgClick_Desciption'],
                 outputCol='features')
train_df = train_assembler.transform(train_df)

IllegalArgumentException: ignored

In [ ]:
# split the dataset
dataset = train_df.select(['features', 'Click'])
train, test = dataset.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression

log_reg = LogisticRegression(labelCol = 'Click').fit(train)

train_pred = log_reg.evaluate(train).predictions

train_pred.filter(train_pred['Click'] == 1).filter(train_pred['prediction'] == 1).select(['Click', 'prediction', 'probability']).show(10, False)

### Model Evaluation

In [ ]:
result_lr = log_reg.evaluate(test).predictions
result_lr.show(3)

In [ ]:
tp_lr = result_lr[(result_lr.label == 1) & (result_lr.prediction == 1)].count()
tn_lr = result_lr[(result_lr.label == 0) & (result_lr.prediction == 1)].count()
fp_lr = result_lr[(result_lr.label == 0) & (result_lr.prediction == 1)].count()
fn_lr = result_lr[(result_lr.label == 1) & (result_lr.prediction == 0)].count()

print('tp is : %f'%(tp_lr))
print('tn is : %f'%(tn_lr))
print('fp is : %f'%(fp_lr))
print('fn is : %f'%(fn_lr))

In [ ]:
# Accuracy
print('test accuracy is : %f'%((tp_lr+tn_lr)/(tp_lr+tn_lr+fp_lr+fn_lr)))

# Recall
print('test accuracy is : %f'%(tp_lr/(tp_lr+fn_lr)))

# Precision
print('test accuracy is : %f'%(tp_lr/(tp_lr+fp_lr)))

In [ ]:
preds_lr = result_lr.select('Click','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['Click']))).collect()

from sklearn.metrics import roc_curve
y_score, y_true = zip(*preds_lr)
fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

## Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Click')
rf_model = rf.fit(train)

In [ ]:
result_rf = rf_model.transform(test).select("Click", "probability").rdd.map(lambda row: (float(row['probability'][1]), float(row['Click']))).collect()
result_rf.show(5)

In [ ]:
tp_rf = result_rf[(result_rf.label == 1) & (result_rf.prediction == 1)].count()
tn_rf = result_rf[(result_rf.label == 0) & (result_rf.prediction == 1)].count()
fp_rf = result_rf[(result_rf.label == 0) & (result_rf.prediction == 1)].count()
fn_rf = result_rf[(result_rf.label == 1) & (result_rf.prediction == 0)].count()

print('tp is : %f'%(tp_rf))
print('tn is : %f'%(tn_rf))
print('fp is : %f'%(fp_rf))
print('fn is : %f'%(fn_rf))

In [ ]:
# Accuracy
print('test accuracy is : %f'%((tp_rf+tn_rf)/(tp_rf+tn_rf+fp_rf+fn_rf)))

# Recall
print('test accuracy is : %f'%(tp_rf/(tp_rf+fn_rf)))

# Precision
print('test accuracy is : %f'%(tp_rf/(tp_rf+fp_rf)))

## Naive Bayes

In [ ]:
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


train_nb_assembler = VectorAssembler(inputCols=['UId',
                        'QId', 'AdId',
                        'KeyId', 'Pos'],
                 outputCol='features')
train_nb_df = train_nb_assembler.transform(training)

In [ ]:
# split the dataset
dataset_nb = train_nb_df.select(['features', 'Click'])
train_nb, test_nb = dataset_nb.randomSplit([0.8, 0.2])

In [ ]:
nb = NaiveBayes(modelType='multinomial')
nbmodel = nb.fit(train_nb)

### Model Evaluation

In [ ]:
result_nb = nbmodel.transform(test_nb)
result_nb.show(5)

In [ ]:
tp_nb = result_nb[(result_nb.label == 1) & (result_nb.prediction == 1)].count()
tn_nb = result_nb[(result_nb.label == 0) & (result_nb.prediction == 1)].count()
fp_nb = result_nb[(result_nb.label == 0) & (result_nb.prediction == 1)].count()
fn_nb = result_nb[(result_nb.label == 1) & (result_nb.prediction == 0)].count()

print('tp is : %f'%(tp_nb))
print('tn is : %f'%(tn_nb))
print('fp is : %f'%(fp_nb))
print('fn is : %f'%(fn_nb))

In [ ]:
# Accuracy
print('test accuracy is : %f'%((tp_nb+tn_nb)/(tp_nb+tn_nb+fp_nb+fn_nb)))

# Recall
print('test accuracy is : %f'%(tp_nb/(tp_nb+fn_nb)))

# Precision
print('test accuracy is : %f'%(tp_nb/(tp_nb+fp_nb)))

In [ ]:
preds_nb = result_nb.select('Click','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['Click']))).collect()

from sklearn.metrics import roc_curve
y_score, y_true = zip(*preds_nb)
fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')